In [1]:
import logging
import os
from datetime import datetime

In [2]:
LOG_FILE= f"{datetime.now().strftime('%Y_%m_%d_%H_%M_%S')}.log"

LOG_FILE

'2023_10_30_19_02_03.log'

In [4]:
os.getcwd()

'd:\\Learning\\INeuron\\Live\\12_2_DS_end_to_end_git_proj\\notebooks'

In [3]:
log_path = os.path.join(os.getcwd(), "logs")
log_path

'd:\\Learning\\INeuron\\Live\\12_2_DS_end_to_end_git_proj\\notebooks\\logs'

In [6]:
from src.DimondPricePrediction.logger import logging

# logg some info
logging.info("Testing out my new logging code")